# Hydroelastic Contact: Embedded Pressure Field
For instructions on how to run these tutorial notebooks, please see the [index](./index.ipynb).

If you are not familiar with Drake's hydroelastic contact, study [hydroelastic_contact_basics.ipynb](./hydroelastic_contact_basics.ipynb). You can also find more information in Hydroelastic Contact User Guide [here.](https://drake.mit.edu/doxygen_cxx/group__hydroelastic__user__guide.html)

In [ ]:
import os
from pathlib import Path

from IPython.display import Code

from pydrake.geometry import StartMeshcat, Rgba
from pydrake.math import RigidTransform
from pydrake.multibody.meshcat import ContactVisualizer, ContactVisualizerParams
from pydrake.multibody.parsing import PackageMap, Parser
from pydrake.multibody.plant import AddMultibodyPlant, MultibodyPlantConfig
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.visualization import ApplyVisualizationConfig, ModelVisualizer, VisualizationConfig

## Start MeshCat

See the section [Viewing models](./authoring_multibody_simulation.ipynb#Viewing-models) in the tutorial [Authoring a Multibody Simulation](./authoring_multibody_simulation.ipynb) for an introduction to MeshCat.

In [ ]:
# Start the visualizer. The cell will output an HTTP link after the execution.
# Click the link and a MeshCat tab should appear in your browser.
meshcat = StartMeshcat()

## Use Experimental VTK file of Embedded Signed Distance Field instead of Body-fitted Tetrahedral Mesh

The finer one (pepper_embsdf.vtk, offset 5mm, grid 1cm) is slower than the body-fitted mesh.
The coarser one (yellow_pepper_Offset1cmGrid2cm_sdfield.vtk, offset 1cm, grid 2cm) is faster.


In [ ]:
# Show the contents of the SDFormat file after replacing the VTK file.
bell_pepper_str = Path(PackageMap().ResolveUrl(
    "package://drake_models/veggies/yellow_bell_pepper_no_stem_low.sdf")).read_text(encoding="utf-8")
bell_pepper_str = bell_pepper_str.replace(
    "package://drake_models/veggies/assets/yellow_bell_pepper_no_stem_low.vtk",
    # Choose one from these three choices: original (300% rtr),
    # finer EmPres (170% rtr), or coarser EmPres (300% rtr).
    # "package://drake_models/veggies/assets/yellow_bell_pepper_no_stem_low.vtk",
    # "package://drake/tutorials/pepper_embsdf.vtk"
    "package://drake/tutorials/yellow_pepper_Offset1cmGrid2cm_sdfield.vtk"
)
Code(bell_pepper_str, language="xml")

In [ ]:
# Visualize the SDFormat file you just defined.
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModels(file_contents=bell_pepper_str, file_type="sdf")
visualizer.Run(loop_once=True)

## Create compliant-hydroelastic table top in URDF

The following URDF file specifies a compliant-hydroelastic box for a table top.  We demonstrate how to set relevant hydroelastic properties in URDF; however, Drake prefers SDFormat to URDF.

Both the `<visual>` and `<collision>` geometries are boxes of the same size.

In the `<drake:proximity_properties>` block, we will set `<drake:hydroelastic_modulus>` to 1e7 Pascals.

We do not specify the inertia matrix of the table top because, in the next section when we set up `Diagram`, we will fix the table top to the world frame. It will not move.

In [ ]:
# Show the contents of the URDF file.
table_top_url = "package://drake_models/dishes/table_top.urdf"
table_top_str = Path(PackageMap().ResolveUrl(table_top_url)).read_text(encoding="utf-8")
Code(table_top_str, language="xml")

In [ ]:
# Visualize the URDF file.
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.AddModels(url=table_top_url)
visualizer.Run(loop_once=True)

## Create Diagram of the scene

The function `add_scene()` below will create a scene using the assets that we created. It will use `Parser` to add the URDF and SDFormat strings into the scene. After this step, the next section will add visualization.

In [ ]:
def add_scene(time_step):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlant(
        MultibodyPlantConfig(
            time_step=time_step,
            discrete_contact_approximation="lagged"),
        builder)
    parser = Parser(plant)

    # Load the assets that we created.
    parser.AddModels(file_contents=bell_pepper_str, file_type="sdf")
    parser.AddModels(url=table_top_url)

    # Weld the table top to the world so that it's fixed during simulation.
    # The top surface passes the world's origin.
    plant.WeldFrames(plant.world_frame(), 
                     plant.GetFrameByName("top_surface"))

    # Finalize the plant after loading the scene.
    plant.Finalize()
    
    # Drop the bell pepper 10 centimeters (measured from its frame's origin) above the ground.
    X_WC = RigidTransform(p=[0, 0, 0.10])
    plant.SetDefaultFreeBodyPose(plant.GetBodyByName("yellow_bell_pepper_no_stem"), X_WC)

    return builder, plant

## Set up visualization

The function `add_viz()` below will create visualization. First we will call `ApplyVisualizationConfig()` to visualize our assets. At this step we will set `publish_contacts=False`, so we can customize contact visualization afterwards. 

To visualize contact result, we will add `ContactVisualizer` with `newtons_per_meter= 20` and `newtons_meters_per_meter= 0.1`. It will draw a red arrow of length 1 meter for each force of 20 newtons and a blue arrow of length 1 meter for each torque of 0.1 newton\*meters. The next section will run the simulation.

In [ ]:
def add_viz(builder, plant):
    ApplyVisualizationConfig(
        builder=builder, meshcat=meshcat,
        config=VisualizationConfig(
                 default_proximity_color=Rgba(r=0.8, g=0, b=0, a=0.5),
                 publish_contacts=False))    
    ContactVisualizer.AddToBuilder(
        builder=builder, plant=plant, meshcat=meshcat,
        params=ContactVisualizerParams(
                 newtons_per_meter=20,
                 newton_meters_per_meter= 0.1))

## Run simulation

We will run the simulation. In MeshCat, the red arrow will represent the force `f`, and the blue arrow will represent the torque `tau`. You should see the contact patch moving around together with the force and torque vectors.

After running the code below, playback with `timeScale` = 0.1 to appreciate the contact dynamics. You should see the force and torque vectors oscillate synchronously with the rocking bell pepper and bowl. See the section *Playback recording of the simulation* in [hydroelastic_contact_basics.ipynb](./hydroelastic_contact_basics.ipynb) for more details.

Currently playing back the simulation will show contact force and torque correctly; however, it does not show contact patch appropriately, which could be confusing. Issue [19142](https://github.com/RobotLocomotion/drake/issues/19142) explains the problem in more details.

In [ ]:
# Clear MeshCat window from the previous blocks.
meshcat.Delete()
meshcat.DeleteAddedControls()

time_step = 1e-2
builder, plant = add_scene(time_step)
add_viz(builder, plant)

diagram = builder.Build()

simulator = Simulator(diagram)

sim_time = 2

meshcat.StartRecording()
simulator.set_target_realtime_rate(10)
simulator.AdvanceTo(sim_time)
meshcat.StopRecording()
meshcat.PublishRecording()

## Download simulation result into a html file for sharing

You can download the simulation result into a self-contained html file, allowing others to playback the simulated results without simulating. The following code prints the URL for downloading. Click on the printed URL to download.

In [ ]:
print(f"{meshcat.web_url()}/download")

## Further reading

* [Hydroelastic Contact User Guide](https://drake.mit.edu/doxygen_cxx/group__hydroelastic__user__guide.html)

* Elandt, R., Drumwright, E., Sherman, M., & Ruina, A. (2019, November). A pressure field model for fast, robust approximation of net contact force and moment between nominally rigid objects. In 2019 IEEE/RSJ International Conference on Intelligent Robots and Systems(IROS) (pp. 8238-8245). IEEE. [link](https://arxiv.org/abs/1904.11433)

* Masterjohn, J., Guoy, D., Shepherd, J., & Castro, A. (2022). Velocity Level Approximation of Pressure Field Contact Patches. IEEE Robotics and Automation Letters 7, no. 4 (2022): 11593-11600. [link](https://arxiv.org/abs/2110.04157v2)

* Elandt, R. (2022, December). Pressure Field Contact. Dissertation. Cornell University. [link](https://ecommons.cornell.edu/handle/1813/112919)